In [2]:
# JesseRuiz_nlp_project_Codeup_2019

In [3]:
from requests import get
import requests
from bs4 import BeautifulSoup
import os
import json
from pprint import pprint
import re

import itertools as it
from typing import List, Dict
import pandas as pd

## Acquire

### General approach to choosing what type of data to scrape from Github: 
#### Use the url: https://github.com/trending

In [3]:
base_url = 'https://github.com/trending'

In [4]:
headers = {'User-Agent': 'student'} # codeup.com doesn't like our default user-agent
response = get(base_url)

In [5]:
print(response.text[:400])







<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="utf-8">
  <link rel="dns-prefetch" href="https://github.githubassets.com">
  <link rel="dns-prefetch" href="https://avatars0.githubusercontent.com">
  <link rel="dns-prefetch" href="https://avatars1.githubusercontent.com">
  <link rel="dns-prefetch" href="https://avatars2.githubusercontent.com">
  <link rel="dns-prefetch" href="htt


In [6]:
pprint(response.text)

('\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '<!DOCTYPE html>\n'
 '<html lang="en">\n'
 '  <head>\n'
 '    <meta charset="utf-8">\n'
 '  <link rel="dns-prefetch" href="https://github.githubassets.com">\n'
 '  <link rel="dns-prefetch" href="https://avatars0.githubusercontent.com">\n'
 '  <link rel="dns-prefetch" href="https://avatars1.githubusercontent.com">\n'
 '  <link rel="dns-prefetch" href="https://avatars2.githubusercontent.com">\n'
 '  <link rel="dns-prefetch" href="https://avatars3.githubusercontent.com">\n'
 '  <link rel="dns-prefetch" href="https://github-cloud.s3.amazonaws.com">\n'
 '  <link rel="dns-prefetch" '
 'href="https://user-images.githubusercontent.com/">\n'
 '\n'
 '\n'
 '\n'
 '  <link crossorigin="anonymous" media="all" '
 'integrity="sha512-3+HOqCwtQal5hOJQ+mdxiq5zmGOTjF6RhjDsPLxbKDYgGlLFeCwzoIanb7j5IiCuXKUqyC2q8FdkC4nmx2P2rA==" '
 'rel="stylesheet" '
 'href="https://github.githubassets.com/assets/frameworks-a2fba223d5af91496cac70d4ec3624df.css" '
 '/>\n'
 '  <link crossor

 'data-hovercard-url="/hovercards?user_id=1961493" '
 'data-octo-click="hovercard-link-click" data-octo-dimensions="link_type:self" '
 'href="/hterkelsen"><img class="avatar mb-1" '
 'src="https://avatars0.githubusercontent.com/u/1961493?s=40&amp;v=4" '
 'width="20" height="20" alt="@hterkelsen" /></a>\n'
 '        </span>\n'
 '\n'
 '      <span class="d-inline-block float-sm-right">\n'
 '        <svg class="octicon octicon-star" viewBox="0 0 14 16" version="1.1" '
 'width="14" height="16" aria-hidden="true"><path fill-rule="evenodd" d="M14 '
 '6l-4.9-.64L7 1 4.9 5.36 0 6l3.6 3.26L2.67 14 7 11.67 11.33 14l-.93-4.74L14 '
 '6z"/></svg>\n'
 '        726 stars today\n'
 '      </span>\n'
 '  </div>\n'
 '</li>\n'
 '\n'
 '              <li class="col-12 d-block width-full py-4 border-bottom" '
 'id="pa-Python">\n'
 '  <div class="d-inline-block col-9 mb-1">\n'
 '    <h3>\n'
 '      <a href="/TheAlgorithms/Python">\n'
 '        <span class="text-normal">TheAlgorithms / </span>Python\n'
 '</a>

 '        </span>\n'
 '\n'
 '      <span class="d-inline-block float-sm-right">\n'
 '        <svg class="octicon octicon-star" viewBox="0 0 14 16" version="1.1" '
 'width="14" height="16" aria-hidden="true"><path fill-rule="evenodd" d="M14 '
 '6l-4.9-.64L7 1 4.9 5.36 0 6l3.6 3.26L2.67 14 7 11.67 11.33 14l-.93-4.74L14 '
 '6z"/></svg>\n'
 '        392 stars today\n'
 '      </span>\n'
 '  </div>\n'
 '</li>\n'
 '\n'
 '              <li class="col-12 d-block width-full py-4 border-bottom" '
 'id="pa-flutter">\n'
 '  <div class="d-inline-block col-9 mb-1">\n'
 '    <h3>\n'
 '      <a href="/flutter/flutter">\n'
 '        <span class="text-normal">flutter / </span>flutter\n'
 '</a>    </h3>\n'
 '  </div>\n'
 '\n'
 '  <div class="float-right">\n'
 '        <a class="btn btn-sm  tooltipped tooltipped-s" aria-label="You must '
 'be signed in to star a repository" rel="nofollow" '
 'data-hydro-click="{&quot;event_type&quot;:&quot;authentication.click&quot;,&quot;payload&quot;:{&quot;location_in_

 '            <svg class="octicon octicon-check select-menu-item-icon" '
 'viewBox="0 0 12 16" version="1.1" width="12" height="16" '
 'aria-hidden="true"><path fill-rule="evenodd" d="M12 5l-8 8-4-4 1.5-1.5L4 '
 '10l6.5-6.5L12 5z"/></svg>\n'
 '            <span data-menu-button-text>Adobe Font Metrics</span>\n'
 '</a>          <a class="select-menu-item" role="menuitemradio" '
 'href="/trending/agda?since=daily">\n'
 '            <svg class="octicon octicon-check select-menu-item-icon" '
 'viewBox="0 0 12 16" version="1.1" width="12" height="16" '
 'aria-hidden="true"><path fill-rule="evenodd" d="M12 5l-8 8-4-4 1.5-1.5L4 '
 '10l6.5-6.5L12 5z"/></svg>\n'
 '            <span data-menu-button-text>Agda</span>\n'
 '</a>          <a class="select-menu-item" role="menuitemradio" '
 'href="/trending/ags-script?since=daily">\n'
 '            <svg class="octicon octicon-check select-menu-item-icon" '
 'viewBox="0 0 12 16" version="1.1" width="12" height="16" '
 'aria-hidden="true"><path fill-ru

 'aria-hidden="true"><path fill-rule="evenodd" d="M12 5l-8 8-4-4 1.5-1.5L4 '
 '10l6.5-6.5L12 5z"/></svg>\n'
 '            <span data-menu-button-text>Coq</span>\n'
 '</a>          <a class="select-menu-item" role="menuitemradio" '
 'href="/trending/cpp-objdump?since=daily">\n'
 '            <svg class="octicon octicon-check select-menu-item-icon" '
 'viewBox="0 0 12 16" version="1.1" width="12" height="16" '
 'aria-hidden="true"><path fill-rule="evenodd" d="M12 5l-8 8-4-4 1.5-1.5L4 '
 '10l6.5-6.5L12 5z"/></svg>\n'
 '            <span data-menu-button-text>Cpp-ObjDump</span>\n'
 '</a>          <a class="select-menu-item" role="menuitemradio" '
 'href="/trending/creole?since=daily">\n'
 '            <svg class="octicon octicon-check select-menu-item-icon" '
 'viewBox="0 0 12 16" version="1.1" width="12" height="16" '
 'aria-hidden="true"><path fill-rule="evenodd" d="M12 5l-8 8-4-4 1.5-1.5L4 '
 '10l6.5-6.5L12 5z"/></svg>\n'
 '            <span data-menu-button-text>Creole</span>\n'
 '</a>

 'aria-hidden="true"><path fill-rule="evenodd" d="M12 5l-8 8-4-4 1.5-1.5L4 '
 '10l6.5-6.5L12 5z"/></svg>\n'
 '            <span data-menu-button-text>NetLinx+ERB</span>\n'
 '</a>          <a class="select-menu-item" role="menuitemradio" '
 'href="/trending/netlogo?since=daily">\n'
 '            <svg class="octicon octicon-check select-menu-item-icon" '
 'viewBox="0 0 12 16" version="1.1" width="12" height="16" '
 'aria-hidden="true"><path fill-rule="evenodd" d="M12 5l-8 8-4-4 1.5-1.5L4 '
 '10l6.5-6.5L12 5z"/></svg>\n'
 '            <span data-menu-button-text>NetLogo</span>\n'
 '</a>          <a class="select-menu-item" role="menuitemradio" '
 'href="/trending/newlisp?since=daily">\n'
 '            <svg class="octicon octicon-check select-menu-item-icon" '
 'viewBox="0 0 12 16" version="1.1" width="12" height="16" '
 'aria-hidden="true"><path fill-rule="evenodd" d="M12 5l-8 8-4-4 1.5-1.5L4 '
 '10l6.5-6.5L12 5z"/></svg>\n'
 '            <span data-menu-button-text>NewLisp</span>\n'
 '</

 '            <span data-menu-button-text>Python traceback</span>\n'
 '</a>          <a class="select-menu-item" role="menuitemradio" '
 'href="/trending/q?since=daily">\n'
 '            <svg class="octicon octicon-check select-menu-item-icon" '
 'viewBox="0 0 12 16" version="1.1" width="12" height="16" '
 'aria-hidden="true"><path fill-rule="evenodd" d="M12 5l-8 8-4-4 1.5-1.5L4 '
 '10l6.5-6.5L12 5z"/></svg>\n'
 '            <span data-menu-button-text>q</span>\n'
 '</a>          <a class="select-menu-item" role="menuitemradio" '
 'href="/trending/qmake?since=daily">\n'
 '            <svg class="octicon octicon-check select-menu-item-icon" '
 'viewBox="0 0 12 16" version="1.1" width="12" height="16" '
 'aria-hidden="true"><path fill-rule="evenodd" d="M12 5l-8 8-4-4 1.5-1.5L4 '
 '10l6.5-6.5L12 5z"/></svg>\n'
 '            <span data-menu-button-text>QMake</span>\n'
 '</a>          <a class="select-menu-item" role="menuitemradio" '
 'href="/trending/qml?since=daily">\n'
 '            <sv

### Create a BeautifulSoup object

In [7]:
soup = BeautifulSoup(response.content, 'html.parser')


In [8]:
soup


<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<link href="https://github.githubassets.com" rel="dns-prefetch"/>
<link href="https://avatars0.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://avatars1.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://avatars2.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://avatars3.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://github-cloud.s3.amazonaws.com" rel="dns-prefetch"/>
<link href="https://user-images.githubusercontent.com/" rel="dns-prefetch"/>
<link crossorigin="anonymous" href="https://github.githubassets.com/assets/frameworks-a2fba223d5af91496cac70d4ec3624df.css" integrity="sha512-3+HOqCwtQal5hOJQ+mdxiq5zmGOTjF6RhjDsPLxbKDYgGlLFeCwzoIanb7j5IiCuXKUqyC2q8FdkC4nmx2P2rA==" media="all" rel="stylesheet">
<link crossorigin="anonymous" href="https://github.githubassets.com/assets/site-148e617d8b0d0d490d3c4030fa22fa7a.css" integrity="sha512-OtdxTByWNKa5cJls+9iFyHPNBFmAN

In [9]:
# We search for all the repositories that have the following class. We should get back 25 objects.
repo = soup.findAll('li', class_='col-12 d-block width-full py-4 border-bottom')
print(type(repo))

<class 'bs4.element.ResultSet'>


In [10]:
print(len(repo))

25


In [11]:
# Show what the repo looks like

for div in repo:
    print(div.text)





microsoft / Terminal
 




      Star




        The new Windows Terminal, and the original Windows console host -- all in the same place!
      




C++



        29,173
      


        2,569
      

          Built by
            







        4,840 stars today
      






jackfrued / Python-100-Days
 




      Star




        Python - 100天从新手到大师
      




HTML



        19,889
      


        6,498
      

          Built by
            







        1,905 stars today
      






kkuchta / css-only-chat
 




      Star




        A truly monstrous async web chat using no JS whatsoever on the frontend
      




Ruby



        3,008
      


        96
      

          Built by
            





        1,312 stars today
      






microsoft / PowerToys
 




      Star




        Windows system utilities to maximize productivity
      




        1,829
      


        58
      

          Built by
            







        984 stars today
      






jo

In [12]:

for div in repo:
    print(div.h3)

<h3>
<a href="/microsoft/Terminal">
<span class="text-normal">microsoft / </span>Terminal
</a> </h3>
<h3>
<a href="/jackfrued/Python-100-Days">
<span class="text-normal">jackfrued / </span>Python-100-Days
</a> </h3>
<h3>
<a href="/kkuchta/css-only-chat">
<span class="text-normal">kkuchta / </span>css-only-chat
</a> </h3>
<h3>
<a href="/microsoft/PowerToys">
<span class="text-normal">microsoft / </span>PowerToys
</a> </h3>
<h3>
<a href="/jolaleye/cssfx">
<span class="text-normal">jolaleye / </span>cssfx
</a> </h3>
<h3>
<a href="/MisterBooo/LeetCodeAnimation">
<span class="text-normal">MisterBooo / </span>LeetCodeAnimation
</a> </h3>
<h3>
<a href="/flutter/flutter_web">
<span class="text-normal">flutter / </span>flutter_web
</a> </h3>
<h3>
<a href="/TheAlgorithms/Python">
<span class="text-normal">TheAlgorithms / </span>Python
</a> </h3>
<h3>
<a href="/hiroppy/fusuma">
<span class="text-normal">hiroppy / </span>fusuma
</a> </h3>
<h3>
<a href="/CyC2018/CS-Notes">
<span class="text-normal"

### After we find the portion of the html that we want, we run through the text with Regex to find the pertinent information. 

### Info gathered: user/repo_name, numb_stars, url_to_repo

In [13]:
repo[0].text

'\n\n\n\nmicrosoft / Terminal\n \n\n\n\n\n      Star\n\n\n\n\n        The new Windows Terminal, and the original Windows console host -- all in the same place!\n      \n\n\n\n\nC++\n\n\n\n        29,173\n      \n\n\n        2,569\n      \n\n          Built by\n            \n\n\n\n\n\n\n\n        4,840 stars today\n      \n\n'

In [14]:
# Show what the user/repo_name is... Create a list to use in the main function.
name_result = []
for div in repo:
    name_result.append(re.findall(r'.*\s/\s.*',div.text))
name_result

[['microsoft / Terminal'],
 ['jackfrued / Python-100-Days'],
 ['kkuchta / css-only-chat'],
 ['microsoft / PowerToys'],
 ['jolaleye / cssfx'],
 ['MisterBooo / LeetCodeAnimation'],
 ['flutter / flutter_web'],
 ['TheAlgorithms / Python'],
 ['hiroppy / fusuma'],
 ['CyC2018 / CS-Notes'],
 ['jaywcjlove / linux-command'],
 ['flutter / flutter'],
 ['996icu / 996.ICU'],
 ['STVIR / pysot'],
 ['minamarkham / formation'],
 ['azl397985856 / leetcode'],
 ['qianguyihao / Web'],
 ['microsoft / react-native-windows'],
 ['sql-machine-learning / sqlflow'],
 ['sabakkps / backslide'],
 ['dgryski / go-perfbook'],
 ['Snailclimb / JavaGuide'],
 ['microsoft / vscode'],
 ['markphelps / flipt'],
 ['teoga / awesome-product-design']]

In [15]:
# Show what the numb_stars, numb_forks, numb_stars_today is... Create a list to use in the main function.
stars_result = []
for div in repo:
    stars_result.append(re.findall(r'.(\d+,?\d*)\s\d*', div.text))
stars_result

[['29,173', '2,569', '4,840'],
 ['19,889', '6,498', '1,905'],
 ['3,008', '96', '1,312'],
 ['1,829', '58', '984'],
 ['1,205', '48', '917'],
 ['21,472', '2,946', '823'],
 ['1,886', '66', '726'],
 ['37,668', '10,695', '416'],
 ['910', '36', '459'],
 ['2018', '61,002', '18,615', '397'],
 ['6,763', '1,539', '392'],
 ['62,984', '6,989', '393'],
 ['243,205', '21,170', '376'],
 ['405', '44', '361'],
 ['961', '68', '346'],
 ['397985856', '5,476', '776', '315'],
 ['3,679', '1,159', '307'],
 ['8,380', '623', '321'],
 ['1,678', '238', '283'],
 ['827', '2', '278'],
 ['6,727', '312', '241'],
 ['39,005', '12,102', '205'],
 ['75,498', '10,290', '207'],
 ['585', '21', '206'],
 ['398', '29', '190']]

In [107]:
# Found on https://madnight.github.io/githut/#/pull_requests/2019/1
langs = ['JavaScript',
'Python',
'Java',
'Go',
'C++',
'Ruby',
'PHP',
'TypeScript',
'C#',
'C',
'Shell',
'Scala',
'Swift',
'Rust',
'Kotlin',
'Objective-C',
'Groovy',
'Elixir',
'Lua',
'DM',
'Perl',
'Dart',
'Clojure',
'PowerShell',
'CoffeeScript',
'Haskell',
'OCaml',
'R',
'Emacs Lisp',
'Vim script',
'Jsonnet',
'Erlang',
'Puppet',
'Julia',
'Smalltalk',
'MATLAB',
'WebAssembly',
'Fortran',
'F#',
'Visual Basic',
'Apex',
'ABAP',
'Vala',
'Nix',
'Crystal',
'Roff',
'Perl 6',
'FreeMarker',
'PureScript',
'Objective-C++',
'HTML',
'CSS',
'Vue']

In [17]:
langs = [element.lower() for element in langs]
langs

['javascript',
 'python',
 'java',
 'go',
 'c++',
 'ruby',
 'php',
 'typescript',
 'c#',
 'c',
 'shell',
 'scala',
 'swift',
 'rust',
 'kotlin',
 'objective-c',
 'groovy',
 'elixir',
 'lua',
 'dm',
 'perl',
 'dart',
 'clojure',
 'powershell',
 'coffeescript',
 'haskell',
 'ocaml',
 'r',
 'emacs lisp',
 'vim script',
 'jsonnet',
 'erlang',
 'puppet',
 'julia',
 'smalltalk',
 'matlab',
 'webassembly',
 'fortran',
 'f#',
 'visual basic',
 'apex',
 'abap',
 'vala',
 'nix',
 'crystal',
 'roff',
 'perl 6',
 'freemarker',
 'purescript',
 'objective-c++',
 'html',
 'css',
 'vue']

In [18]:
# Looks for the programming language, which is one of many choices, like Python, C++, Javascript...
# Based on a hardcoded list of programming languages (some of the most popular), the query checks 
# both lists and returns the element that appears in both lists. Essentially, finding the langauge.

repo_list = re.findall(r'.+', repo[0].text.lower())
repo_list

['microsoft / terminal',
 ' ',
 '      star',
 '        the new windows terminal, and the original windows console host -- all in the same place!',
 '      ',
 'c++',
 '        29,173',
 '      ',
 '        2,569',
 '      ',
 '          built by',
 '            ',
 '        4,840 stars today',
 '      ']

In [19]:
list(set(repo_list) & set(langs))

['c++']

In [75]:
list(set(repo_list) & set(langs))

[]

In [96]:
# Function shows a list of labels (Programming Language) of a repository. 
# The function takes a BeautifulSoup ResultSet and returns a list.
def get_labels(bs4resultset):
    labels = []
    for div in bs4resultset:
        repo_list = re.findall(r'.+', div.text.lower())
        lang = list(set(repo_list) & set(langs))
        if lang != []:
            labels.append(lang[0])
        else:
            labels.append('na')
    return labels


In [97]:
test = get_labels(repo)
test

['c++',
 'html',
 'ruby',
 'na',
 'vue',
 'java',
 'dart',
 'python',
 'css',
 'java',
 'html',
 'dart',
 'rust',
 'na',
 'shell',
 'javascript',
 'javascript',
 'c#',
 'go',
 'na',
 'na',
 'java',
 'typescript',
 'go',
 'na']

In [58]:
for div in repo:
    print(re.findall(r'.*\s/\s.*',div.text)[0])

microsoft / Terminal
jackfrued / Python-100-Days
kkuchta / css-only-chat
microsoft / PowerToys
jolaleye / cssfx
MisterBooo / LeetCodeAnimation
flutter / flutter_web
TheAlgorithms / Python
hiroppy / fusuma
CyC2018 / CS-Notes
jaywcjlove / linux-command
flutter / flutter
996icu / 996.ICU
STVIR / pysot
minamarkham / formation
azl397985856 / leetcode
qianguyihao / Web
microsoft / react-native-windows
sql-machine-learning / sqlflow
sabakkps / backslide
dgryski / go-perfbook
Snailclimb / JavaGuide
microsoft / vscode
markphelps / flipt
teoga / awesome-product-design


In [104]:
# Function shows what the user/repo_name and the label (Programming Language) is... 
# Creates a list of lists.

def repo_name_and_label(bs4resultset):
    name_label = []
    for div in bs4resultset:
        one = []
        one.append(re.findall(r'.*\s/\s.*',div.text)[0])
        lang = list(set(re.findall(r'.+', div.text.lower())) & set(langs))
        if lang != []:
            one.append(lang[0])
        else:
            one.append('na')
        name_label.append(one)
    return name_label

In [105]:
repo_name_and_label(repo)

[['microsoft / Terminal', 'c++'],
 ['jackfrued / Python-100-Days', 'html'],
 ['kkuchta / css-only-chat', 'ruby'],
 ['microsoft / PowerToys', 'na'],
 ['jolaleye / cssfx', 'vue'],
 ['MisterBooo / LeetCodeAnimation', 'java'],
 ['flutter / flutter_web', 'dart'],
 ['TheAlgorithms / Python', 'python'],
 ['hiroppy / fusuma', 'css'],
 ['CyC2018 / CS-Notes', 'java'],
 ['jaywcjlove / linux-command', 'html'],
 ['flutter / flutter', 'dart'],
 ['996icu / 996.ICU', 'rust'],
 ['STVIR / pysot', 'na'],
 ['minamarkham / formation', 'shell'],
 ['azl397985856 / leetcode', 'javascript'],
 ['qianguyihao / Web', 'javascript'],
 ['microsoft / react-native-windows', 'c#'],
 ['sql-machine-learning / sqlflow', 'go'],
 ['sabakkps / backslide', 'na'],
 ['dgryski / go-perfbook', 'na'],
 ['Snailclimb / JavaGuide', 'java'],
 ['microsoft / vscode', 'typescript'],
 ['markphelps / flipt', 'go'],
 ['teoga / awesome-product-design', 'na']]

## Acquire

### General approach to choosing what type of data to scrape from Github: 
#### Use the url: https://github.com/search?o=desc&q=stars%3A%3E1&s=updated&type=Repositories
### This page shows the most recently updates repos on Github

In [27]:
base_url = 'https://github.com/search?o=desc&q=stars%3A%3E1&s=updated&type=Repositories'

In [28]:
headers = {'User-Agent': 'student'} # codeup.com doesn't like our default user-agent
response = get(base_url)

In [29]:
print(response.text[:400])



<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="utf-8">
  <link rel="dns-prefetch" href="https://github.githubassets.com">
  <link rel="dns-prefetch" href="https://avatars0.githubusercontent.com">
  <link rel="dns-prefetch" href="https://avatars1.githubusercontent.com">
  <link rel="dns-prefetch" href="https://avatars2.githubusercontent.com">
  <link rel="dns-prefetch" href="https:/


In [30]:
pprint(response.text)

('\n'
 '\n'
 '<!DOCTYPE html>\n'
 '<html lang="en">\n'
 '  <head>\n'
 '    <meta charset="utf-8">\n'
 '  <link rel="dns-prefetch" href="https://github.githubassets.com">\n'
 '  <link rel="dns-prefetch" href="https://avatars0.githubusercontent.com">\n'
 '  <link rel="dns-prefetch" href="https://avatars1.githubusercontent.com">\n'
 '  <link rel="dns-prefetch" href="https://avatars2.githubusercontent.com">\n'
 '  <link rel="dns-prefetch" href="https://avatars3.githubusercontent.com">\n'
 '  <link rel="dns-prefetch" href="https://github-cloud.s3.amazonaws.com">\n'
 '  <link rel="dns-prefetch" '
 'href="https://user-images.githubusercontent.com/">\n'
 '\n'
 '\n'
 '\n'
 '  <link crossorigin="anonymous" media="all" '
 'integrity="sha512-3+HOqCwtQal5hOJQ+mdxiq5zmGOTjF6RhjDsPLxbKDYgGlLFeCwzoIanb7j5IiCuXKUqyC2q8FdkC4nmx2P2rA==" '
 'rel="stylesheet" '
 'href="https://github.githubassets.com/assets/frameworks-a2fba223d5af91496cac70d4ec3624df.css" '
 '/>\n'
 '  <link crossorigin="anonymous" media="

 '            \n'
 '          >HTML</option>\n'
 '          <option\n'
 '            '
 'value="/search?l=Shell&amp;q=stars%3A%3E1&amp;s=updated&amp;type=Repositories"\n'
 '            \n'
 '          >Shell</option>\n'
 '      </select>\n'
 '    </div>\n'
 '\n'
 '\n'
 '<!-- Put sort options in filter area on small screens  -->\n'
 '  <div class="px-2 pt-2 d-flex d-md-none flex-items-center">\n'
 '    <label for="codesearch_sort_repos" class="col-3 text-small '
 'px-2">Sort</label>\n'
 '    <select id="codesearch_sort_repos" class="col-9 form-select mr-2" '
 'data-pjax>\n'
 '          <option\n'
 '            '
 'value="/search?o=desc&amp;q=stars%3A%3E1&amp;s=&amp;type=Repositories"\n'
 '            \n'
 '          >Best match</option>\n'
 '          <option\n'
 '            '
 'value="/search?o=desc&amp;q=stars%3A%3E1&amp;s=stars&amp;type=Repositories"\n'
 '            \n'
 '          >Most stars</option>\n'
 '          <option\n'
 '            '
 'value="/search?o=asc&amp;q=stars%3A%

 '        <p class="f6 text-gray mr-3 mb-0 mt-2">\n'
 '          AGPL-3.0 license\n'
 '        </p>\n'
 '\n'
 '        <p class="f6 text-gray mr-3 mb-0 mt-2">\n'
 '          Updated <relative-time datetime="2019-05-09T21:55:18Z">May 9, '
 '2019</relative-time>\n'
 '        </p>\n'
 '\n'
 '          <a class="muted-link f6 mt-2" '
 'href="/movim/movim/issues?q=label%3A%22help+wanted%22+is%3Aissue+is%3Aopen">\n'
 '            7 issues\n'
 '            need help\n'
 '          </a>\n'
 '\n'
 '    </div>\n'
 '  </div>\n'
 '\n'
 '  <div class="flex-shrink-0 col-6 col-md-4 pt-2 pr-md-3 d-flex">\n'
 '    <div class="text-gray flex-auto min-width-0">\n'
 '        <div class="mr-3">\n'
 '          <span class=" ">\n'
 '  <span class="repo-language-color" style="background-color: '
 '#4F5D95"></span>\n'
 '  <span itemprop="programmingLanguage">PHP</span>\n'
 '</span>\n'
 '\n'
 '        </div>\n'
 '    </div>\n'
 '\n'
 '      <div class="pl-2 pl-md-0 text-right flex-auto min-width-0">\n'
 '      

### Create a BeautifulSoup object

In [33]:
soup = BeautifulSoup(response.content, 'html.parser')


In [34]:
soup


<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<link href="https://github.githubassets.com" rel="dns-prefetch"/>
<link href="https://avatars0.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://avatars1.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://avatars2.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://avatars3.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://github-cloud.s3.amazonaws.com" rel="dns-prefetch"/>
<link href="https://user-images.githubusercontent.com/" rel="dns-prefetch"/>
<link crossorigin="anonymous" href="https://github.githubassets.com/assets/frameworks-a2fba223d5af91496cac70d4ec3624df.css" integrity="sha512-3+HOqCwtQal5hOJQ+mdxiq5zmGOTjF6RhjDsPLxbKDYgGlLFeCwzoIanb7j5IiCuXKUqyC2q8FdkC4nmx2P2rA==" media="all" rel="stylesheet">
<link crossorigin="anonymous" href="https://github.githubassets.com/assets/site-148e617d8b0d0d490d3c4030fa22fa7a.css" integrity="sha512-OtdxTByWNKa5cJls+9iFyHPNBFmAN

### This looks for the repo class, which is structured differently than the TRENDING page which we used before...
### Each page contains 10 repos... 

In [35]:
# We search for all the repositories that have the following class. We should get back 25 objects.
repo2 = soup.findAll('li', class_='repo-list-item d-flex flex-column flex-md-row flex-justify-start py-4 public source')
print(type(repo))

<class 'bs4.element.ResultSet'>


In [36]:
print(len(repo2))

10


In [37]:
# Show what the repo looks like

for div in repo2:
    print(div.text)




maxmind/MaxMind-DB


        Spec and test data for the MaxMind DB file format
      


          Updated May 9, 2019








Perl






          128
        






kate-orlova/google-places-in-sitecore


        Google Places in Sitecore
      


          MIT license
        

          Updated May 9, 2019








C#






          9
        






nicksergeant/dotfiles


        nicksergeant's dotfiles
      


        zsh


        arch-linux


        i3wm




          MIT license
        

          Updated May 9, 2019








Shell






          9
        






CloudCannon/jekyll-theme-cloudcannon


🎨 Company Jekyll theme
      


          Updated May 9, 2019








JavaScript






          2
        






ThexXTURBOXx/studip-app-uni-passau


        Stud.IP App for the Uni Passau
      


          GPL-3.0 license
        

          Updated May 9, 2019








Java






          3
        






jlctools/evo


        Evo C++ Library
      


          BSD-2-Cl

### Now that we have an entirely different HTML text structure, we have to write new regex to capture the relevant information.

### TODO: find out how to crawl across multiple pages to scrape these pages. Aiming for 150 repos, which is 15 pages.

In [38]:
# Function shows what the user/repo_name and the label (Programming Language) is... 
# Creates a list of lists.

def repo_name_and_label(bs4resultset):
    name_label = []
    for div in bs4resultset:
        one = []
        one.append(re.findall(r'.*\s/\s.*',div.text)[0])
        lang = list(set(re.findall(r'.+', div.text.lower())) & set(langs))
        if lang != []:
            one.append(lang[0])
        else:
            one.append('na')
        name_label.append(one)
    return name_label

In [39]:
# Finds the user/repo_name...
for div in repo2:
#     print(div.text)
    print(re.findall(r'.*/.*',div.text)[0])

maxmind/MaxMind-DB
kate-orlova/google-places-in-sitecore
nicksergeant/dotfiles
CloudCannon/jekyll-theme-cloudcannon
ThexXTURBOXx/studip-app-uni-passau
jlctools/evo
FriendsOfCake/awesome-cakephp
tigergraph/ecosys
r-lib/styler
movim/movim


In [54]:
# Finds the number of stars...
for div in repo2:
#     print(div.text)
    print(re.findall(r'(\d*)', div.text))

['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '9', '', '', '2019', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '128', '', '', '', '', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''

## Prepare

## Clean

## Explore

## Visualize

## Model

## Predict